In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split 

 

# Last inn datasettet 

df = pd.read_csv("trondheim_vaerdata_full.csv", parse_dates = ['Tidspunkt'])
print(df.columns)

df['year'] = df['Tidspunkt'].dt.year 

df['day_of_year'] = df['Tidspunkt'].dt.dayofyear 

 

# Håndter manglende verdier 

df['mean(air_temperature P1D)'] = df['mean(air_temperature P1D)'].interpolate() 

df = df.dropna(subset=['mean(air_temperature P1D)'])  # Dropper rader uten temperatur 

 

# Visualisering 1: Linjediagram (temperatur over tid) 

plt.figure(figsize=(12, 4)) 

sns.lineplot(data=df, x='Tidspunkt', y='mean(air_temperature P1D)') 

plt.title("Daglig gjennomsnittstemperatur (2020–2025)") 

plt.xlabel("Dato") 

plt.ylabel("Temperatur (°C)") 

plt.tight_layout() 

plt.show() 

 

# Visualisering 2: Søylediagram (månedlig nedbør) 

monthly_precip = df.resample('M', on='Tidspunkt').sum(numeric_only=True) 

plt.figure(figsize=(10, 4)) 

monthly_precip['sum(precipitation_amount P1D)'].plot(kind='bar') 

plt.title("Månedlig nedbør (mm)") 

plt.ylabel("Nedbør (mm)") 

plt.tight_layout() 

plt.show() 

 

# Visualisering 3: Scatterplot (temperatur vs. nedbør) 

plt.figure(figsize=(8, 5)) 

sns.scatterplot(data=df, x='sum(precipitation_amount P1D)', y='mean(air_temperature P1D)', alpha=0.5) 

plt.title("Temperatur vs. nedbør") 

plt.xlabel("Nedbør (mm)") 

plt.ylabel("Temperatur (°C)") 

plt.tight_layout() 

plt.show() 

 

# ---- Modelltrening ---- 

# Velg features og target 

features = df[['year', 'day_of_year']] 

target = df['mean(air_temperature P1D)'] 

 

# Tren på data før 2025, test på 2025 

X_train = features[df['year'] < 2025] 

X_test = features[df['year'] == 2025] 

y_train = target[df['year'] < 2025] 

y_test = target[df['year'] == 2025] 

 

# Tren modellen 

model = LinearRegression() 

model.fit(X_train, y_train) 

 

# Evaluer modellen 

y_pred = model.predict(X_test) 

print("MAE:", mean_absolute_error(y_test, y_pred)) 

print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred))) 

 

# ---- Prediksjon for 2026 ---- 

future_2026 = pd.DataFrame({ 

    'year': [2026] * 365, 

    'day_of_year': list(range(1, 366))  # 2026 er skuddår 

}) 

 

future_2026['predicted_temperature'] = model.predict(future_2026) 

 

# Visualiser prediksjonen 

plt.figure(figsize=(10, 4)) 

plt.plot(future_2026['day_of_year'], future_2026['predicted_temperature']) 

plt.title("Forventet daglig temperatur i 2026 (lineær regresjon)") 

plt.xlabel("Dag i året") 

plt.ylabel("Predikert temperatur (°C)") 

plt.tight_layout() 

plt.show()

ModuleNotFoundError: No module named 'sklearn'